In [1]:
import re, os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.messages import HumanMessage

from dotenv import load_dotenv
#os.environ['OPENAI_API_KEY'] = "YOUR_API_KEY"

## 1. Agent 설정
LLM, Temperature, 프롬프트, 페르소나 등

In [8]:
llm = ChatOpenAI(model="gpt-4o-mini",
                    temperature=0,
                    request_timeout=60,
                    api_key=os.environ["OPENAI_API_KEY"])

system_prompt = '''Y
You are a logical reasoning assistant.  
You will be shown a short story as a list of sentences and then a question.  
Your task is to answer using exactly one English word naming a location (e.g., “desk”, “bookshelf”).  
Do not add any extra words, punctuation, or explanation—just the single location word.
'''

integrated_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt.strip()),
    ("human", """
[Story]
{Story}

[Question]
{Question}

""")
])

## 1. 질문 하나만 테스트 해보기

In [9]:
story = '''
1. Oliver entered the porch.
2. Owen entered the porch.
3. Abigail entered the porch.
4. The broccoli is in the bucket.
5. Abigail exited the porch.
6. Owen exited the porch.
7. Owen hates the peach
8. Oliver moved the broccoli to the pantry.
'''
qa_list = [
    {'type': 'reality', 'question':'Where is the broccoli really?', 'answer':'pantry'},
    {'type':'memory', 'question':'Where was the broccoli at the beginning?', 'answer':'bucket'},
    {'type':'first_order', 'question':'Where will Oliver look for the broccoli?', 'answer':'pantry'},
    {'type':'first_order', 'question':'Where will Abigail look for the broccoli?', 'answer':'bucket'},
    {'type':'second_order', 'question':'Where does Oliver think that Abigail searches for the broccoli?', 'answer':'pantry'},
    {'type':'second_order', 'question':'Where does Abigail think that Oliver searches for the broccoli?', 'answer':'pantry'},
]

In [10]:
for i, qa in enumerate(qa_list, 1):
    response = llm.invoke(integrated_prompt.format_messages(
        Story=story.strip(), Question=qa['question']
    ))
    answer = response.content.strip()
    print(f"\033[94mQuestion {i} ({qa['type']}): ){qa['question']}\033[0m")
    print(f"  Model answer: {answer}, Correct answer: {qa['answer']}")

Question 1 (reality): )Where is the broccoli really?
  Model answer: pantry, Correct answer: pantry
Question 2 (memory): )Where was the broccoli at the beginning?
  Model answer: bucket, Correct answer: bucket
Question 3 (first_order): )Where will Oliver look for the broccoli?
  Model answer: pantry, Correct answer: pantry
Question 4 (first_order): )Where will Abigail look for the broccoli?
  Model answer: pantry, Correct answer: bucket
Question 5 (second_order): )Where does Oliver think that Abigail searches for the broccoli?
  Model answer: pantry, Correct answer: pantry
Question 6 (second_order): )Where does Abigail think that Oliver searches for the broccoli?
  Model answer: pantry, Correct answer: pantry
